In [2]:
%matplotlib notebook
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d as m3d
import matplotlib.pyplot as plt
import numpy as np
import itertools
import colorsys # Convert RGB values to hsv

# Function to convert RGB values to YCbCr
def rgb_to_ycbcr(R, G, B): # in (0,255) range
    Y = .299 * R + .587 * G + .114 * B
    Cb = 128 - .168736 * R - .331364 * G + .5 * B
    Cr = 128 + .5 * R - .418688 * G - .081312 * B
#     Y = .299 * R + .287 * G + .11 * B
#     Cr = R - Y
#     Cb = B - Y
    return Y, Cb, Cr

# Construct RGB values tuple
rgb = (range(94, 256), range(40, 256), range(0, 256))

# Find all RGB combinations
comb = list(itertools.product(*rgb))

# Convert RGB values to HSV and YCbCr and add them to the end of the RGB tuple
rgbhsvycbcr = []
for i in comb:
    addToRgb = i + colorsys.rgb_to_hsv(i[0], i[1], i[2]) + rgb_to_ycbcr(i[0], i[1], i[2])
    rgbhsvycbcr.append(addToRgb)

filteredSkinTones = []
# testRgbhsvycbcr = [(95, 40, 22, 0.16666666666666666, 0.5, 95, 54.16499999999999, 108.71552, 157.12624)]
# rgbhsvycbcr = [r, g, b, h, s, v, y, cb, cr]
for i in rgbhsvycbcr:
    if ((i[0] > i[1]) and (i[0] > i[2]) and (i[0] - i[1] > 15) and \
    (i[3] <= 50 and i[3] >= 0 and i[4] <= 0.68 and i[4] > 0.1)) or \
    ((i[0] > i[1]) and (i[0] > i[2]) and (i[0] - i[1] > 15) and \
    (i[8] > 135 and i[7] > 85 and i[6] > 8) and \
    (i[8] <= (1.5862 * i[7]) + 34) and \
    (i[8] >= (0.3448 * i[7]) + 76.2069) and \
    (i[8] >= (-4.5652 * i[7]) + 234.5652) and \
    (i[8] <= (-1.15 * i[7]) + 301.75) and \
    (i[8] <= (-2.2857 * i[7]) + 432.85)):
        filteredSkinTones.append(i)

# Get only RGB values from rgbhsvycbcr tuples
allSkinTones = [i[0:3] for i in filteredSkinTones]

# Construct a dataframe of RGB and Hexcodes
allSkinTonesDf = pd.DataFrame(allSkinTones)
allSkinTonesDf['HexCodes'] = ['#%02x%02x%02x' % i for i in allSkinTones]
allSkinTonesDf = allSkinTonesDf.rename(index=str, columns={0: "R", 1: "G", 2: "B"})
allSkinTonesDf.head(5)

# Create params for chart
R = allSkinTonesDf['R'].astype('float64')
G = allSkinTonesDf['G'].astype('float64')
B = allSkinTonesDf['B'].astype('float64')
allSkinTonesColor = allSkinTonesDf['HexCodes']

In [3]:
productsDf = pd.read_excel('../data/input.xlsx')
radius = 20

productSkinTones = []
for index, row in productsDf.iterrows():
    r = productsDf['R'][index]
    g = productsDf['G'][index]
    b = productsDf['B'][index]
    rRange = range(r-radius, r+radius if r+radius < 255 else 255)
    gRange = range(g-radius, g+radius if g+radius < 255 else 255)
    bRange = range(b-radius, b+radius if b+radius < 255 else 255)
    rgb = (rRange, gRange, bRange)
    comb = list(itertools.product(*rgb))
    productSkinTones.append(comb)

productSkinTones = list(set([i for sublist in productSkinTones for i in sublist]))
productSkinTonesDf = pd.DataFrame(productSkinTones)
productSkinTonesDf['HexCodes'] = ['#%02x%02x%02x' % i for i in productSkinTones]
productSkinTonesDf = productSkinTonesDf.rename(index=str, columns={0: "R", 1: "G", 2: "B"})

# Create params for chart
x = productSkinTonesDf['R'].astype('float64')
y = productSkinTonesDf['G'].astype('float64')
z = productSkinTonesDf['B'].astype('float64')
productColor = list(productSkinTonesDf['HexCodes'])
productCoveredRange = [i/40 for i in list(productsDf['Skin Tones Covered Per Product'])]

In [4]:
# # Construct 3D Scatter Plot
# ax = m3d.Axes3D(plt.figure(figsize=(8, 7)))
# # ax.scatter3D(x, y, z, c=productColor, s=productCoveredRange, depthshade=False)
# ax.scatter3D(R, G, B, c=allSkinTonesColor)
# ax.set_xlim3d(0, 255)
# ax.set_ylim3d(0, 255)
# ax.set_zlim3d(0, 255)
# ax.set_xlabel('R', fontsize=20)
# ax.set_ylabel('G', fontsize=20)
# ax.set_zlabel('B', fontsize=20)
# # ax.azim = 315
# ax.elev = 20
# plt.show()

In [5]:
# plt.savefig('../image/uncovered_skin_tones.png')

In [6]:
print len(allSkinTones), len(set(productSkinTones))
uncoveredSkinTones = list(set(allSkinTones) - set(allSkinTones).intersection(set(productSkinTones)))
coveredSkinTones = list(set(allSkinTones).intersection(set(productSkinTones)))
print len(coveredSkinTones), len(uncoveredSkinTones)
uncoveredSkinTones[0:3]

901095 1168263
777416 123679


[(154, 131, 132), (153, 46, 27), (216, 167, 172)]

In [14]:
rValue = zip(*uncoveredSkinTones)
for i in rValue:
    avg = np.mean(i)
    med = np.median(i)
    print avg, med

160.81577309 155.0
114.580510839 108.0
92.2290768845 93.0
